In [ ]:
import pickle
import os
import pandas as pd
import numpy as np
np.random.seed(42)
from sklearn.model_selection import train_test_split as splt
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.callbacks import EarlyStopping
import seaborn as sns
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [ ]:
df = pd.read_csv("Data/new_wScore.csv")

In [ ]:
df.sort_values("visit")
df = df.select_dtypes(exclude=['object', 'datetime64'])
df = df.drop(labels = ['SCORE','ana_fis:smoking_recod', 'lab:glucose', 'lab:calculated_ldl', 'lab:total_cholesterol'], axis=1)
df = df.fillna(-1).replace(-1, np.nan).replace("-1", np.nan).replace(-1.0, np.nan)
df = df.dropna(how='any', axis=1)

In [ ]:
df.head()

In [ ]:
d = {i: df.loc[df.subject_id == i, df.columns] for i in range(df.subject_id.iat[-1]+1)}
d = {dx: d[dx] for dx in d if d[dx].shape[0] != 0}
#X = np.array([np.array(values)[:-1,:-1] for key, values in d.items()])
#X = np.array([X[:,:,i]/X[:,:,i].max() for i in range(X.shape[2])])
#print(X.shape)
#X = np.swapaxes(X, 0, 1)
#X = np.swapaxes(X, 1, 2)
#y = np.array([np.array(values)[-1,-1] for key, values in d.items()])
#yy = np.zeros((y.shape[0], 4))
#for i,yyy in enumerate(y):
#    if(yyy < 0.01): # Low
#       yy[i][0] = 1
#    elif(yyy < 0.02): # Medium
#        yy[i][1] = 1
#    elif(yyy < 0.05): # High
#        yy[i][2] = 1
#    else: # Very High (aka Dead)
#        yy[i][3] = 1 

In [ ]:
X1 = []
y1 = []
X2 = []
y2 = []
X3 = []
y3 = []
for key, value in d.items():
    row1 = value.iloc[[0]]
    row2 = value.iloc[[1]]
    row3 = value.iloc[[2]]
    row4 = value.iloc[[3]]
    y1.append(int(row2.ScoreClass))
    X1.append(row1.drop('ScoreClass', axis=1).values[0])
    y1.append(int(row3.ScoreClass))
    X1.append(row2.drop('ScoreClass', axis=1).values[0])
    y1.append(int(row4.ScoreClass))
    X1.append(row3.drop('ScoreClass', axis=1).values[0])
    y2.append(int(row3.ScoreClass))
    X2.append(row1.drop('ScoreClass', axis=1).values[0])
    y2.append(int(row4.ScoreClass))
    X2.append(row2.drop('ScoreClass', axis=1).values[0])
    y3.append(int(row4.ScoreClass))
    X3.append(row1.drop('ScoreClass', axis=1).values[0])

In [ ]:
X1 = np.array(X1)
X2 = np.array(X2)
X3 = np.array(X3)
y1 = np.array(y1)
y2 = np.array(y2)
y3 = np.array(y3)

In [ ]:
print(X1.shape)

In [ ]:
print(X.shape)
model = Sequential()
model.add(LSTM(16, input_shape = X.shape[1:], return_sequences=True, activation='relu'))
model.add(Dropout(0.2))
model.add(LSTM(8, return_sequences=False, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(8, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(4, activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam')
print(y, y.shape)
print(X, X.shape)

In [ ]:
X_tr, X_ts, y_tr, y_ts = splt(X, yy, test_size=0.25, random_state=42)
es = EarlyStopping(patience = 20, restore_best_weights = True)
history = model.fit(X_tr, y_tr, validation_data=(X_ts, y_ts), epochs = 340, callbacks = [es])